IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M4\NO_FAULT4


In [3]:
guasto=0

VOLO m4 FAULT 10%

In [4]:
rcou_m4_fault10 = pd.read_csv("RCOU.csv")
rcou_m4_fault10 = rcou_m4_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)
rcou_m4_fault10 = rcou_m4_fault10[((rcou_m4_fault10['C9'] >= 1350) & (rcou_m4_fault10['C10'] >= 1350) & (rcou_m4_fault10['C11'] >= 1350) & (rcou_m4_fault10['C12'] >= 1350) & (rcou_m4_fault10['C13'] >= 1350) & (rcou_m4_fault10['C14']>= 1350))]
rcou_m4_fault10=rcou_m4_fault10.reset_index(drop=True)
display(rcou_m4_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76367829,1406,1351,1366,1393,1388,1370
1,76399898,1372,1427,1439,1358,1409,1391
2,76420523,1389,1437,1444,1382,1417,1411
3,76439487,1471,1375,1402,1448,1446,1403
4,76471372,1411,1475,1464,1423,1428,1459
...,...,...,...,...,...,...,...
5094,136813708,1657,1369,1584,1468,1512,1545
5095,136823888,1633,1404,1581,1472,1479,1575
5096,136833858,1603,1445,1592,1458,1455,1594
5097,136848913,1631,1407,1600,1450,1499,1558


In [5]:
min=rcou_m4_fault10['TimeUS'][0]
max=rcou_m4_fault10['TimeUS'][len(rcou_m4_fault10)-1]
print(max)
print(min)

136861088
76367829


In [6]:
xkf1_m4_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault10 = xkf1_m4_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault10 = xkf1_m4_fault10[((xkf1_m4_fault10['TimeUS'] >= min) & (xkf1_m4_fault10['TimeUS'] <= max	))]
xkf1_m4_fault10 = xkf1_m4_fault10.reset_index(drop=True)
print(xkf1_m4_fault10)

         TimeUS  Roll  Pitch     Yaw
0      76375630  1.69  -1.08   16.30
1      76386278  1.67  -1.10   16.29
2      76397328  1.62  -1.13   16.29
3      76407963  1.54  -1.13   16.28
4      76418382  1.51  -1.14   16.29
...         ...   ...    ...     ...
5474  136811131 -1.72   0.04  355.34
5475  136821892 -1.71   0.04  355.34
5476  136831987 -1.74   0.05  355.34
5477  136846510 -1.83   0.11  355.33
5478  136859050 -1.73   0.04  355.36

[5479 rows x 4 columns]


In [7]:
att_m4_fault10 = pd.read_csv("ATT.csv")
att_m4_fault10 = att_m4_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault10 = att_m4_fault10[((att_m4_fault10['TimeUS'] >= min) & (att_m4_fault10['TimeUS'] <= max	))]
att_m4_fault10=att_m4_fault10.reset_index(drop=True)
print(att_m4_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      76377698     0.18  1.69      0.14  -1.08   16.23   16.30
1      76388276     0.18  1.67      0.15  -1.10   16.23   16.29
2      76399032     0.17  1.62      0.15  -1.13   16.23   16.29
3      76409551     0.17  1.54      0.16  -1.13   16.23   16.28
4      76419805     0.17  1.51      0.16  -1.14   16.23   16.29
...         ...      ...   ...       ...    ...     ...     ...
5475  136812901     0.75 -1.72     -1.40   0.04  354.13  355.35
5476  136823075     0.79 -1.71     -1.40   0.04  354.13  355.35
5477  136833108     0.84 -1.74     -1.41   0.05  354.13  355.35
5478  136848053     0.87 -1.83     -1.42   0.11  354.13  355.34
5479  136860307     0.90 -1.73     -1.44   0.04  354.13  355.37

[5480 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m4_fault10)>len(xkf1_m4_fault10):
    to_add=att_m4_fault10[len(xkf1_m4_fault10):]
    att_m4_fault10=att_m4_fault10[:len(xkf1_m4_fault10)]

if len(xkf1_m4_fault10)>len(att_m4_fault10):
    to_add=xkf1_m4_fault10[len(att_m4_fault10):]

for idx,i in enumerate(att_m4_fault10['Roll']):
    if(xkf1_m4_fault10['Roll'][idx] != i):
        att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Pitch']):
    if(xkf1_m4_fault10['Pitch'][idx] != i):
        att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Yaw']):
    if(xkf1_m4_fault10['Yaw'][idx] != i):
        att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault10=pd.concat([att_m4_fault10,to_add])

print(att_m4_fault10)

C:\Users\mgale\AppData\Local\Temp\ipykernel_26172\3465845594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_26172\3465845594.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_26172\3465845594.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      76377698     0.18  1.690      0.14 -1.080   16.23   16.300
1      76388276     0.18  1.670      0.15 -1.100   16.23   16.290
2      76399032     0.17  1.620      0.15 -1.130   16.23   16.290
3      76409551     0.17  1.540      0.16 -1.130   16.23   16.280
4      76419805     0.17  1.510      0.16 -1.140   16.23   16.290
...         ...      ...    ...       ...    ...     ...      ...
5475  136812901     0.75 -1.715     -1.40  0.040  354.13  355.345
5476  136823075     0.79 -1.725     -1.40  0.045  354.13  355.345
5477  136833108     0.84 -1.785     -1.41  0.080  354.13  355.340
5478  136848053     0.87 -1.780     -1.42  0.075  354.13  355.350
5479  136860307     0.90 -1.730     -1.44  0.040  354.13  355.370

[5480 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault10 = esc_0_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault10 = esc_0_m4_fault10[((esc_0_m4_fault10['TimeUS'] >= min) & (esc_0_m4_fault10['TimeUS'] <= max))]
esc_0_m4_fault10=esc_0_m4_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault10 = esc_1_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault10 = esc_1_m4_fault10[((esc_1_m4_fault10['TimeUS'] >= min) & (esc_1_m4_fault10['TimeUS'] <= max))]
esc_1_m4_fault10=esc_1_m4_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault10 = esc_2_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault10 = esc_2_m4_fault10[((esc_2_m4_fault10['TimeUS'] >= min) & (esc_2_m4_fault10['TimeUS'] <= max))]
esc_2_m4_fault10=esc_2_m4_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault10 = esc_3_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault10 = esc_3_m4_fault10[((esc_3_m4_fault10['TimeUS'] >= min) & (esc_3_m4_fault10['TimeUS'] <= max))]
esc_3_m4_fault10=esc_3_m4_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault10 = esc_4_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault10 = esc_4_m4_fault10[((esc_4_m4_fault10['TimeUS'] >= min) & (esc_4_m4_fault10['TimeUS'] <= max))]
esc_4_m4_fault10=esc_4_m4_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault10 = esc_5_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault10 = esc_5_m4_fault10[((esc_5_m4_fault10['TimeUS'] >= min) & (esc_5_m4_fault10['TimeUS'] <= max))]
esc_5_m4_fault10=esc_5_m4_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault10 = imu_0_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault10 = imu_0_m4_fault10[((imu_0_m4_fault10['TimeUS'] >= min) & (imu_0_m4_fault10['TimeUS'] <= max))]

imu_1_m4_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault10 = imu_1_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault10 = imu_1_m4_fault10[((imu_1_m4_fault10['TimeUS'] >= min) & (imu_1_m4_fault10['TimeUS'] <= max))]

imu_2_m4_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault10 = imu_2_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault10 = imu_2_m4_fault10[((imu_2_m4_fault10['TimeUS'] >= min) & (imu_2_m4_fault10['TimeUS'] <= max))]

imu_m4_fault10 = pd.concat((imu_0_m4_fault10, imu_1_m4_fault10, imu_2_m4_fault10))
imu_m4_fault10=imu_m4_fault10.groupby(imu_m4_fault10.TimeUS, as_index=False).mean()

display(imu_m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76369982,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346
1,76372073,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593
2,76374333,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186
3,76377446,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467
4,76380512,0.025879,0.046393,0.026676,-0.255138,-0.149288,-9.542381
...,...,...,...,...,...,...,...
20834,136835711,0.067217,-0.045699,-0.004597,-0.369982,0.197985,-9.371787
20835,136841561,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052
20836,136847878,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913
20837,136853859,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault10 = pd.merge_ordered(imu_m4_fault10,att_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_0_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_1_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_2_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_3_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_4_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_5_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
m4_fault10 = pd.merge_ordered(m4_fault10, rcou_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m4_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

5480
5987
6500
7010
7512
8016
8514


In [13]:
m4_fault10 = m4_fault10[~m4_fault10.TimeUS.isin(index_to_remove)]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"] - m4_fault10.iloc[0]["TimeUS"]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"].astype(int)
m4_fault10["Guasto"] = guasto

In [14]:
from datetime import timedelta
m4_fault10=m4_fault10.reset_index(drop=True)
m4_fault10['TimeUS'] = pd.to_datetime(m4_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0
1,00:00:00.002091,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0
2,00:00:00.004351,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0
3,00:00:00.007464,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0
4,00:00:00.010530,0.025879,0.046393,0.026676,-0.255138,-0.149288,-9.542381,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20834,00:01:00.465729,0.067217,-0.045699,-0.004597,-0.369982,0.197985,-9.371787,0.84,-1.785,-1.41,...,1.75,5839.0,3.95,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0
20835,00:01:00.471579,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052,0.84,-1.785,-1.41,...,1.75,5839.0,3.95,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0
20836,00:01:00.477896,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.84,-1.785,-1.41,...,1.75,5839.0,3.95,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0
20837,00:01:00.483877,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})','guasto'])
    for i in range(int(len(m4_fault10)/350)):
        V1=m4_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2,'guasto':guasto}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_26172\947090789.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_26172\947090789.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),guasto_x,...,guasto_y,<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),guasto
0,0.0,-0.047938,0.010689,0.236867,0.113827,88.0,0.0,5.731168,2.761700,0.0,...,0.0,3.315812,0.609686,-1.641786,3.406253,127.0,125.0,54.307084,6.138906,0.0
1,1.0,-0.048007,0.010660,0.246309,0.113731,95.0,0.0,5.511450,2.879252,0.0,...,0.0,3.317812,0.609357,-1.638812,3.408153,127.0,125.0,54.343508,5.566326,0.0
2,2.0,-0.047872,0.010636,0.249296,0.113570,100.0,0.0,5.355139,3.068190,0.0,...,0.0,3.319802,0.609023,-1.635796,3.410041,127.0,125.0,54.361851,4.954289,0.0
3,3.0,-0.047682,0.010619,0.247721,0.113414,101.0,0.0,5.210325,3.276285,0.0,...,0.0,3.321780,0.608683,-1.632740,3.411918,127.0,125.0,54.362542,5.127294,0.0
4,4.0,-0.047667,0.010589,0.256602,0.113277,92.0,0.0,4.995397,3.421549,0.0,...,0.0,3.323746,0.608336,-1.629644,3.413783,127.0,125.0,54.346011,5.565556,0.0
5,5.0,-0.047647,0.010560,0.265344,0.113138,91.0,0.0,4.774767,3.567930,0.0,...,0.0,3.325702,0.607985,-1.626509,3.415636,127.0,125.0,54.312685,5.981911,0.0
6,6.0,-0.047557,0.010533,0.271141,0.112982,88.0,0.0,4.571989,3.738553,0.0,...,0.0,3.327647,0.607627,-1.623335,3.417479,127.0,125.0,54.262992,6.376296,0.0
7,7.0,-0.047549,0.010503,0.280158,0.112849,103.0,0.0,4.333195,3.879106,0.0,...,0.0,3.329581,0.607264,-1.620123,3.419309,129.0,127.0,54.197356,6.748799,0.0
8,8.0,-0.047543,0.010474,0.289218,0.112717,98.0,0.0,4.087332,4.018057,0.0,...,0.0,3.331504,0.606895,-1.616873,3.421129,127.0,125.0,54.116201,7.099639,0.0
9,9.0,-0.047440,0.010449,0.294217,0.112562,0.0,99.0,4.190881,3.870315,0.0,...,0.0,3.333417,0.606522,-1.613586,3.422938,127.0,125.0,54.019948,7.429160,0.0


In [18]:
path_file = path_file.replace(r"m4\FAULT_m4_10", "")
os.chdir(path_file)
m4_fault10.to_csv('m4_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M4\NO_FAULT4
